# LOスペクトラムサンプラー
このjupyter notebookはwaveshaperにてパススルー時のスペクトラムをサンプルすることを目的とする。
- optSpectrum.npy
- optFreq.npy

計 2ファイルを保存し、各種のスペクトラム最適化アルゴリズムにて、waveshaperによる加工前のスペクトラムを参考に補償することを目的とする。

In [1]:
import vxi11
from WSMethods import getWSrange
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [2]:
class wrapper:
    def __init__(self, instrument: vxi11.Instrument):
        self.instr = instrument

    def __enter__(self):
        return self.instr

    def __exit__(self, ex_type, ex_value, trace):
        self.instr.close()


def setOSArange(osa_ip: str, startFreq, endFreq):
    """startFreq < endFreq
    """
    assert startFreq <= endFreq
    with wrapper(vxi11.Instrument(osa_ip)) as instr:
        instr.write(f':SENSe:WAVelength:STARt {float(endFreq)}THZ') # short wavelength
        instr.write(f':SENSe:WAVelength:STOP {float(startFreq)}THZ') # long wavelength

def getOSA(osa_ip: str, trace_name):
    c = 299792458

    with wrapper(vxi11.Instrument(osa_ip)) as instr:# 真空の光速
        instr.write(':SENSE:CORRECTION:RVELOCITY:MEDIUM VACUUM')

    with wrapper(vxi11.Instrument(osa_ip)) as instr:
        instr.write(':UNIT:X WAVelength')
    # 基本的にどのように設定しようが、波長単位でデータが帰ってくるので関係なし
    

    #光スペアナのx軸（波長）取得
    with wrapper(vxi11.Instrument(osa_ip) ) as instr:
        wavelength = instr.ask(':TRACe:X? '+trace_name)
        wavelength = wavelength.split(',')
        wavelength = [float(ii) for ii in wavelength]
        wavelength = np.array(wavelength)# m

    freq = c/wavelength/1E12 # THz

    #光スペアナのy軸（dBm/nm）取得
    with wrapper(vxi11.Instrument(osa_ip)) as instr:
        pnbw = instr.ask(':CALCULATE:MATH:TRF:PNBW:BAND?')#正規化帯域幅

    with wrapper(vxi11.Instrument(osa_ip)) as instr:
        data = instr.ask(':TRACe:Y:PDENsity? '+trace_name+',0.1nm')
        data = data.split(',')
        data = [float(ii) for ii in data]
        data = np.array(data)

    optWavelength = wavelength
    optFreq = freq
    optSpectrum = data
    
    
    return optWavelength, optFreq, optSpectrum


In [3]:
ws_ip  = '169.254.6.8'
osa_ip = "169.254.6.9"
trace_name = "TRA"
c = 299792458 # m/s



# ルーチン
startFreq, endFreq = getWSrange(ws_ip)
setOSArange(osa_ip, startFreq, endFreq)
optWavelength, optFreq, optSpectrum = getOSA(osa_ip, trace_name)
np.save('optFreq.npy', optFreq)
np.save('optSpectrum.npy', optSpectrum)
#np.save('optWavelength', optWavelength)

In [4]:
fig = make_subplots(rows=1, cols=2, subplot_titles=("power", "phase"))
go_element = lambda x,y,name: go.Scatter(x=x,y=y,mode='lines',name=name)
fig.add_trace(go_element(optFreq, optSpectrum, 'optSpectrum'), row=1, col=1)
fig.update_xaxes(title_text="Freq", row=1, col=1)
fig.update_yaxes(title_text="power", row=1, col=1)
fig.update_xaxes(title_text="Freq", row=1, col=2)
fig.update_yaxes(title_text="Phase", row=1, col=2)